In [23]:
#Importando as bibliotecas

# Data Analysis Tools
import pandas as pd
import numpy as np

# Visualization Tools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Machine Learning Tools
import sklearn
import sklearn.mixture as mix
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

plt.rcParams['figure.figsize'] = (10, 10)

def get_sensor_names(tag_names):
    """ Get tagnames starting with sensor.
    
    :param tag_names: Input time series data frame
    
    :return list of string tag names starting with sensor.
    """
    
    return [tag_name for tag_name in tag_names if tag_name.startswith('sensor')]

In [39]:
#Tag Names que foram explicadas no arquivo Challenge Data description 2016
tag_names = ['unit','cycle_num', 'setting1','setting2',
         'setting3', 'sensor1', 'sensor2',
         'sensor3', 'sensor4', 'sensor5', 'sensor6',
         'sensor7', 'sensor8', 'sensor9', 'sensor10',
         'sensor11', 'sensor12', 'sensor13', 'sensor14',
         'sensor15', 'sensor16', 'sensor17', 'sensor18',
         'sensor19', 'sensor20', 'sensor21']

sensor_columns = get_sensor_names(tag_names)
train_data = pd.read_csv('train.txt', sep=' ', header=None, names=tag_names)

print "O dataset de treino tem {} linhas de dados e {} atributos.".format(*train_data.shape)

O dataset de treino tem 45918 linhas de dados e 26 atributos.


In [41]:
train_data.head(100)

unit  cycle_num  setting1  setting2  setting3  sensor1  sensor2  \
1 1    10.0047     0.2501      20.0    489.05    604.13  1499.45  1309.95   
  2     0.0015     0.0003     100.0    518.67    642.13  1584.55  1403.96   
  3    34.9986     0.8401      60.0    449.44    555.42  1368.17  1122.49   
  4    20.0031     0.7005       0.0    491.19    607.03  1488.44  1249.18   
  5    42.0041     0.8405      40.0    445.00    549.52  1354.48  1124.32   
  6    20.0032     0.7017       0.0    491.19    607.37  1480.46  1258.90   
  7    41.9998     0.8400      40.0    445.00    549.57  1354.43  1131.44   
  8     0.0011     0.0000     100.0    518.67    642.08  1589.55  1407.59   
  9     0.0011     0.0020     100.0    518.67    642.70  1586.18  1399.01   
  10   42.0066     0.8400      40.0    445.00    549.83  1353.19  1125.09   
  11   25.0051     0.6200      80.0    462.54    537.41  1260.50  1051.56   
  12   35.0029     0.8413      60.0    449.44    555.85  1360.54  1130.69   
  13   42.0029     0.8409      40.0    445.00    549.91  1350.90  1123.53   
  14   25.0073     0.6203      80.0    462.54    537.22  1262.80  1051.01   
  15    9.9988     0.2500      20.0    489.05    604.63  1497.87  1302.51   
  16    9.9987     0.2500      20.0    489.05    604.61  1500.47  1311.39   
  17    0.0003     0.0000     100.0    518.67    642.30  1584.57  1410.11   
  18   10.0066     0.2507      20.0    489.05    604.23  1502.57  1304.56   
  19   25.0018     0.6200      80.0    462.54    536.66  1255.21  1053.69   
  20    0.0004     0.0017     100.0    518.67    642.31  1581.33  1399.44   
  21   42.0030     0.8404      40.0    445.00    549.12  1349.16  1117.33   
  22    0.0004     0.0000     100.0    518.67    642.34  1589.25  1401.52   
  23   20.0025     0.7011       0.0    491.19    607.10  1482.68  1252.00   
  24   34.9987     0.8400      60.0    449.44    555.19  1359.28  1131.95   
  25   42.0066     0.8401      40.0    445.00    549.43  1355.26  1117.68   
  26    0.0003     0.0004     100.0    518.67    642.90  1591.72  1409.43   
  27   25.0031     0.6206      80.0    462.54    536.89  1267.93  1042.92   
  28    0.0020     0.0000     100.0    518.67    641.60  1590.43  1402.64   
  29   25.0026     0.6200      80.0    462.54    536.59  1256.53  1048.20   
  30   42.0008     0.8400      40.0    445.00    549.77  1350.63  1133.09   
...        ...        ...       ...       ...       ...      ...      ...   
  71   35.0076     0.8400      60.0    449.44    555.99  1359.31  1130.70   
  72   42.0060     0.8403      40.0    445.00    549.93  1351.67  1126.20   
  73   41.9980     0.8400      40.0    445.00    549.14  1351.31  1120.20   
  74   35.0054     0.8400      60.0    449.44    555.13  1360.96  1127.22   
  75   20.0072     0.7020       0.0    491.19    607.25  1482.81  1248.39   
  76   20.0026     0.7009       0.0    491.19    607.14  1483.93  1250.76   
  77   35.0055     0.8406      60.0    449.44    555.89  1366.31  1128.67   
  78   35.0024     0.8412      60.0    449.44    555.12  1365.46  1121.76   
  79   41.9996     0.8415      40.0    445.00    549.51  1353.91  1117.26   
  80   42.0029     0.8413      40.0    445.00    549.91  1361.86  1126.25   
  81   35.0014     0.8401      60.0    449.44    555.67  1366.70  1130.51   
  82   35.0008     0.8416      60.0    449.44    555.58  1361.55  1130.46   
  83   25.0027     0.6206      80.0    462.54    536.73  1258.32  1045.61   
  84   42.0025     0.8403      40.0    445.00    549.78  1350.97  1129.54   
  85   35.0049     0.8400      60.0    449.44    556.10  1364.31  1129.86   
  86   35.0043     0.8400      60.0    449.44    555.71  1365.64  1130.92   
  87   25.0046     0.6214      80.0    462.54    536.81  1265.37  1046.02   
  88   41.9995     0.8400      40.0    445.00    550.04  1348.00  1130.76   
  89   10.0057     0.2512      20.0    489.05    604.38  1500.89  1309.16   
  90    0.0012     0.0000     100.0    518.67    642.29  1587.81  1406.04   
  91  

In [32]:
train_data.describe()

,unit,cycle_num,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
count,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,...,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,45918.000000,0.0,0.0
mean,24.008881,0.571251,49.172438,472.861677,579.596770,1419.947816,1205.669194,8.029878,11.596773,282.469820,...,8065.548831,9.332433,0.023354,348.293915,2228.342654,97.741699,20.776822,12.466282,NaN,NaN
std,14.793676,0.310935,32.503659,26.460179,37.391068,106.286960,119.550936,3.627125,5.451857,146.539301,...,84.964099,0.751834,0.004722,27.837602,145.693883,5.378969,9.903697,5.941954,NaN,NaN
min,0.000000,0.000000,0.000000,445.000000,535.650000,1245.430000,1029.490000,3.910000,5.710000,136.750000,...,7851.590000,8.325300,0.020000,303.000000,1915.000000,84.930000,10.230000,6.126600,NaN,NaN
25%,10.004500,0.250600,20.000000,445.000000,549.580000,1352.910000,1124.010000,3.910000,5.720000,139.662500,...,8061.230000,8.676925,0.020000,331.000000,2212.000000,100.000000,10.840000,6.504600,NaN,NaN
50%,25.001500,0.700000,40.000000,462.540000,555.985000,1369.140000,1138.870000,7.050000,9.030000,194.630000,...,8081.475000,9.314500,0.020000,335.000000,2223.000000,100.000000,14.870000,8.925200,NaN,NaN
75%,41.998100,0.840000,80.000000,491.190000,607.370000,1499.697500,1307.437500,10.520000,15.490000,394.100000,...,8126.380000,9.390000,0.030000,369.000000,2324.000000,100.000000,28.470000,17.084200,NaN,NaN
max,42.008000,0.842000,100.000000,518.670000,644.430000,1614.660000,1442.360000,14.620000,21.610000,555.720000,...,8274.880000,11.082600,0.030000,398.000000,2388.000000,100.000000,39.330000,23.590500,NaN,NaN
